# Deep Q Network

## 1. Setup

In [34]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

env = gym.make('CartPole-v0').unwrapped

state = env.reset()
print(state)

action = env.action_space.sample()
print(action)

n_action = env.action_space.n
n_state = env.observation_space.shape[0]
print(n_state)
print(n_action)

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion() # interactive mode on: 交互模式

[-0.00984778 -0.04756129  0.00088064 -0.01369826]
1
4
2


## 2. Replay Memory

In [25]:
Experience = namedtuple('Experience', ('state', 'action', 'next_state', 'reward', 'terminal'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = -1

    def push(self, *args):
        self.position = (self.position + 1) % self.capacity
        # if len(self.memory) < self.capacity:
        #     self.memory.append(None)
        self.memory[self.position] = Transition(*args) ## append a new experience

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self): ## len(experience)
        return len(self.memory)


experience_pool = ReplayMemory(10000) #initialize memory pool

## 3. Q-Network

In [27]:
class NN():
    def __init__(self, input_size, output_size):
        super(NN, self).__init__()
        self.model = nn.Sequential(
        nn.Linear(input_size, 100),
        nn.ReLU(),
        nn.Linear(100, 100),
        nn.ReLU(),
        nn.Linear(100, output_size))

policy_net = NN(n_state,n_action).model #initialize nn models
target_net = NN(n_state,n_action).model
target_net.load_state_dict(policy_net.state_dict()) ## copy policy to target

<All keys matched successfully>

## 4. Hyperparameters


In [28]:
BATCH_SIZE = 128 #batch-train
GAMMA = 0.9 #reward-discount
EPS = 0.1 #epsilon-greedy
TARGET_UPDATE = 20 #policy to target

optimizer = optim.SGD(policy_net.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss()

## 5. Optimize



In [ ]:
def optimize_model():
    if len(experience_pool) < BATCH_SIZE:
        return #not enough experience

    experiences = experience_pool.sample(BATCH_SIZE)
    experiences_batch = Experience(*zip(*experiences)) #experiences of batches

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = loss_fn(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## 6. Exploration

In [ ]:
    # if sample > eps_threshold:
    #     with torch.no_grad():
    #         # t.max(1) will return largest column value of each row.
    #         # second column on max result is index of where max element was
    #         # found, so we pick action with the larger expected reward.
    #         return policy_net(state).max(1)[1].view(1, 1)
    # else:
def greedy_action(state):
    pass

def sample_action(state):
    if np.random.uniform(0, 1) < (1.0 - EPS): #choose a by policy-NN
        action = self.predict(obs) #greedy
    else:
        action = env.action_space.sample() #random action

        return torch.tensor([[action]], dtype=torch.int)

## 7. Train Loop

In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    # Initialize the environment and state
    state = env.reset() # s
    for t in count():
        # Select and perform an action
        action = select_action(state)
        _, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        last_screen = current_screen
        current_screen = get_screen()
        if not done:
            next_state = current_screen - last_screen
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
